In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from pprint import pprint

from functools import partial

import numpy as np
from sklearn.preprocessing import scale
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import matplotlib.animation as ani

import mlgrad.models as models
import mlgrad.regr as regr
import mlgrad.cls as cls
import mlgrad.pca as pca
import mlgrad.plots as plots
import mlgrad.funcs as funcs
import mlgrad.funcs2 as funcs2
import mlgrad.loss as loss
import mlgrad.gd as gd
import mlgrad.smooth as smooth
import mlgrad.inventory as inventory
from mlgrad.af import averaging_function
import mlgrad.avragg as avragg

import rampy
import pybaselines
import random

from irsa import io
from irsa.preprocess import despike, utils
import irsa.widgets as widgets
import irsa.spectra as spectra
import irsa.io as io

import scipy.special as special

import copy

import ipywidgets
from IPython.display import display

import sys
print(sys.version)

%matplotlib widget
plt.ion()

3.13.2 (main, Mar 29 2025, 10:04:43) [GCC 14.2.0]


In [ ]:
# path = f"/home/intellimath/repos/spectra/bacteria1024"
# path = f"/home/intellimath/repos/spectra/bacteria25_6"
# path = f"/home/intellimath/repos/spectra/mechanical_lysis"
# path = f"/home/intellimath/repos/spectra/thermal_lysis"
path = f"/home/intellimath/repos/spectra/bacteria25_7"
options = {}
dd = {}
# widgets.load_spectra(path, dd, options)

In [ ]:
# dd = io.load_spectra(path, {"вид_бактерий":"STR", "отсечки_по_молекулярной_массе":"less30more10"})
# dd = io.load_spectra(path, {"штамм_бактерий":["36", "95", "B", "C"], "тип_измерения_спектров":["SE"],
#                             "отсечки_по_молекулярной_массе": ["less100"],})
# dd = io.load_spectra(path, {"штамм_бактерий":["36", "95"], #"A", "B", "C", "D"], 
#                              "отсечки_по_молекулярной_массе": "less100",
#                              "тип_измерения_спектров":"SE"})
# dd = io.load_spectra(path, {})
dd = io.load_spectra(path, {"вид_бактерий":["SA", "SH", "SS"], 
                            "номер_цикла":["7"], "номер_эксперимента_в_цикле":["5"],
                            "отсечки_по_молекулярной_массе": ["more100"],
                           })
print(list(dd.keys()))

d_keys = list(sorted(dd.keys()))
sel_i = 0
sel = ipywidgets.Dropdown(options=d_keys,  description="Experiment:", rows=len(dd)+1)
sel.layout.width="50%"

In [ ]:
for i, key in enumerate(dd):
    entry1 = dd[key]
    entry1.allign_bottom()
    # entry1.scale_by_max(scale=10.0)
    entry1.crop(100)

In [ ]:
@ipywidgets.interact(key=sel)
def plot_spectra_for_key(key):
    entry = dd[key]
    entry.plot_spectra()

In [ ]:
# @ipywidgets.interact(key=sel)
# def check_spectra_for_key(key):
#     entry = dd[key]
#     entry.check_spectra()

In [ ]:
# for i, key in enumerate(dd):
#     entry = dd[key]
#     entry.exclude_checked()    

In [ ]:
# @ipywidgets.interact(key=sel)
# def plot_spectra_for_key(key):
#     entry = dd[key]
#     entry.plot_spectra()

In [ ]:
d2 = spectra.SpectraCollection()
for i, key in enumerate(dd):
    entry2 = dd[key]
    d2[key] = entry2.robust_averaging(tau=3.0)

In [ ]:
for key in d2:
    entry = d2[key]
    entry.allign_bottom()
    entry.scale_by_max(scale=10.0)
    # entry.apply_func(y_func=np.log, x_func=np.log, b=1)


In [ ]:
d2.save("/home/intellimath/work/spectra/spectra_7_5_SA_SH_SS", "origin")

In [6]:
d3 = spectra.SpectraCollection()
d3.load("/home/intellimath/work/spectra/spectra_7_5_SA_SH_SS", "origin")
for key in d3:
    sp = d3[key]
    print(key, sp.x.shape, sp.y.shape)
    # print(sp.attrs)


20250401_SS_17_no_date_more100_7_5_1_SE_no_date_1_H2O(HPLC)_да_ (924,) (21, 924)
20250401_SS_17_no_date_more100_7_5_1_SE_no_date_3_H2O(HPLC)_да_ (924,) (24, 924)
20250401_SH_15_no_date_more100_7_5_1_SE_no_date_3_H2O(HPLC)_да_ (924,) (20, 924)
20250401_SA_95_R_more100_7_5_1_SE_no_date_1_H2O(HPLC)_да_ (924,) (20, 924)
20250401_SA_95_R_more100_7_5_1_SE_no_date_2_H2O(HPLC)_да_ (924,) (20, 924)
20250401_SH_15_no_date_more100_7_5_1_SE_no_date_2_H2O(HPLC)_да_ (924,) (20, 924)
20250401_SS_17_no_date_more100_7_5_1_SE_no_date_2_H2O(HPLC)_да_ (924,) (21, 924)
20250401_SA_95_R_more100_7_5_1_SE_no_date_3_H2O(HPLC)_да_ (924,) (20, 924)
20250401_SH_15_no_date_more100_7_5_1_SE_no_date_1_H2O(HPLC)_да_ (924,) (20, 924)


In [7]:
d_keys = list(sorted(d3.keys()))
sel_i = 0
sel = ipywidgets.Dropdown(options=d_keys,  description="Experiment:", rows=len(d3)+1)
sel.layout.width="50%"

In [ ]:
@ipywidgets.interact(key=sel)
def plot_spectra_for_key(key):
    entry = d3[key]
    entry.plot_spectra()

In [ ]:
# for key in d2:
#     entry = d2[key]
#     entry.smooth(tau=10.0, func2=funcs.RectExp(w=10.0, p=1.0).evaluate_array)

In [8]:
@ipywidgets.interact(key=sel)
def select_windows_for_key(key):
    entry = d3[key]
    # entry.windows = [(1560.0,2690.0), (3010.0,3790.0)]
    # entry.smooth_by_windows(windows=entry.windows, tau=10)
    entry.select_windows()

interactive(children=(Dropdown(description='Experiment:', layout=Layout(width='50%'), options=('20250401_SA_95…

In [ ]:
# for key in d2:
#     entry = d2[key]
#     entry.smooth_by_windows(windows=entry.windows, tau=10)

In [ ]:
# @ipywidgets.interact(key=sel)
# def plot_spectra_for_key(key):
#     entry = d2[key]
#     entry.plot_spectra()

In [ ]:
# plt.figure(figsize=(14,5))
# for j,key in enumerate(d2):
#     # print(key)
#     entry = d2[key]
#     xs, ys = entry.x, entry.y
#     if "95" in key:
#         color="b"
#     elif "36" in key:
#         color="r"
#     plt.plot(xs, np.mean(ys, axis=0), label=key[1:], c=color)
# plt.legend()
# plt.show()    

In [ ]:
@ipywidgets.interact(key=sel)
def select_baseline_param_for_key(key):
    entry = d3[key]
    entry.select_baseline_param(tau2=1.0e5, bs_scale=2.5, d=2, func2_mode="e",
                           ww=10, tau_smooth=100, n_iter=100, tol=1.0e-6,
                           func=funcs.Step(0.1), tau_z=0,
                           func2=inventory.relative_abs_max,
                           override_tau2=True,
                          )


In [ ]:
for key in d3:
    entry = d3[key]
    entry.subtract_baselines(
        d=2, func2_mode="e",
        func=funcs.Step(0.1),
        func2=inventory.relative_abs_max,        
    )
    entry.replace_spectra_with_corrected()

In [ ]:
# def func2(E):
#     E_abs = abs(E)
#     return E_abs / np.max(E_abs)

@ipywidgets.interact(key=sel)
def plot_spectra_for_key(key):
    entry = d3[key]
    entry.select_baselines(tau2=1.0e5, tau1=0, bs_scale=2.5, d=2, func2_mode="e",
                           ww=10, tau_smooth=100, n_iter=100, tol=1.0e-6,
                           func=funcs.Step(0.1),
                           func2=inventory.relative_abs_max,        
                           override_tau2=True
                          )


In [ ]:
# for key in d2:
#     entry = d2[key]
#     entry.subtract_selected_baselines()

In [ ]:
d3.save("/home/intellimath/work/spectra/spectra_7_5_SA_SH_SS", "ys-bs-1")

In [ ]:
# @ipywidgets.interact(key=sel)
# def plot_spectra_for_key(key):
#     entry = d2[key]
#     entry.plot_corrected_spectra()

In [ ]:
# for i, key in enumerate(d2):
#     entry = d2[key]
#     entry.replace_spectra_with_corrected()

In [ ]:
for key in d2:
    entry = d2[key]
    entry.allign_bottom()
    entry.scale_by_max(scale=10.0)

In [ ]:
for i, key in enumerate(d3):
    entry = d3[key]
    plt.figure(figsize=(12,4))
    plt.title(key)
    xs = entry.x
    Ys = entry.y
    for ys in Ys:
        plt.plot(xs, ys, linewidth=0.5)
    plt.hlines(0, min(xs), max(xs), colors='k', linestyles='--')
    plt.xlim(min(xs)-20, max(xs))
        
    plt.minorticks_on()
    plt.tight_layout()
    # plt.legend()
    plt.show()


In [ ]:
from irsa.analytics.pca import pca_compare_2d

ys36 = np.concatenate(
    tuple(d2.select_y_by_attr_value("штамм_бактерий", "36")), axis=0)
ys95 = np.concatenate(
    tuple(d2.select_y_by_attr_value("штамм_бактерий", "95")), axis=0)
ysB = np.concatenate(
    tuple(d2.select_y_by_attr_value("штамм_бактерий", "B")), axis=0)
ysC = np.concatenate(
    tuple(d2.select_y_by_attr_value("штамм_бактерий", "C")), axis=0)

print("36:", ys36.shape, "95:", ys95.shape)

pca_compare_2d(ys36, ys95, "36", "95", 0.975)
pca_compare_2d(ys36, ysB, "36", "B", 0.975)
pca_compare_2d(ys95, ysC, "95", "C", 0.975)
pca_compare_2d(ysB, ysC, "B", "C", 0.975)


In [ ]:
import mlgrad.models as models
import mlgrad.funcs as funcs
import mlgrad.funcs2 as funcs2
import mlgrad.loss as loss

import mlgrad.cls as cls
import mlgrad.af as af

zs36 = np.ones(ys36.shape[0], "d")
zs95 = -np.ones(ys95.shape[0], "d")
zsB = np.ones(ysB.shape[0], "d")
zsC = -np.ones(ysC.shape[0], "d")

# print(ys36.shape, zs36.shape)
# print(ys95.shape, zs95.shape)

ys = np.concatenate((ys36,ys95), axis=0)
zs = np.concatenate((zs36,zs95), axis=0)
ys = np.ascontiguousarray(ys, dtype="d")
zs = np.ascontiguousarray(zs, dtype="d")

ysBC = np.concatenate((ysB,ysC), axis=0)
zsBC = np.concatenate((zsB,zsC), axis=0)
ysBC = np.ascontiguousarray(ysBC, dtype="d")
zsBC = np.ascontiguousarray(zsBC, dtype="d")

In [ ]:

# print(ys.shape, zs.shape)

# body = models.FFNetworkModel()
# body.add(models.LinearLayer(ys.shape[1], 100))
# body.add(models.ScaleLayer(funcs.Logistic(1.0), 100))
# head = models.LinearModel(100)
# nnmod = models.FFNetworkFuncModel(head, body)
# nnmod.allocate()
# nnmod.init_param()

mod = models.SigmaNeuronModel(funcs.Sigmoidal(1.0), ys.shape[1])
mod.allocate()
mod.init_param()

In [ ]:
alg = cls.classification_as_regr(ys, zs, mod, regnorm=funcs2.AbsoluteNorm(),
                                 normalizer=gd.Masked(mod.n_param, 1.0e-5), 
                                 h=0.001, tau=0.01, n_iter=5000)

plt.figure(figsize=(10,3))
plt.plot(alg.lvals)
plt.show()

from sklearn.metrics import accuracy_score
print(accuracy_score(zs, np.sign(mod.evaluate(ys))))

In [ ]:
param = np.asarray(mod.param[1:])
print(param[0])
param_p = param.copy()
param_m = param.copy()
param_p[param < 0] = 0
param_m[param > 0] = 0
plt.figure(figsize=(10,5))
plt.plot(ys36.mean(axis=0), label="36")
plt.plot(ys95.mean(axis=0), label="95")
plt.vlines(range(ys.shape[1]), 10*param_m, 10*param_p, color='k')
plt.show()

In [ ]:
plots.plot_cls_function(mod, ys, zs)

In [ ]:
mod2 = models.SigmaNeuronModel(funcs.Sigmoidal(1.0), ysBC.shape[1])
mod2.allocate()
mod2.init_param()

In [ ]:
alg2 = cls.classification_as_regr(ysBC, zsBC, mod2, regnorm=funcs2.AbsoluteNorm(),
                                 normalizer=gd.Masked(mod.n_param, 1.0e-5), 
                                 h=0.01, tau=0.01, n_iter=5000)

plt.figure(figsize=(10,3))
plt.plot(alg2.lvals)
plt.show()

from sklearn.metrics import accuracy_score
print(accuracy_score(zsBC, np.sign(mod2.evaluate(ysBC))))

In [ ]:
param2 = np.asarray(mod2.param[1:])
param2_p = param2.copy()
param2_m = param2.copy()
param2_p[param2 < 0] = 0
param2_m[param2 > 0] = 0
plt.figure(figsize=(12,5))
plt.plot(ysBC.mean(axis=0))
plt.vlines(range(ysBC.shape[1]), 10*param2_m, 10*param2_p, color='k')
plt.show()

In [ ]:
plots.plot_cls_function(mod2, ysBC, zsBC)

In [ ]:
# from mlgrad.af import averaging_function
# from mlgrad.pca import robust_location, find_robust_pc_all
# from irsa.analytics.pca import robust_pca_compare_2d

# # wma = averaging_function("WMZ", kwds={"alpha":3.5})

# robust_pca_compare_2d(ys36, ys95, "36", "95", kind="WZ")

# robust_pca_compare_2d(ys95, ys36, "95", "36", kind="WZ")

In [ ]:
# from mlgrad.pca import location, find_pc_all, find_pc_l1_all, find_pc_smoothed_all

# plt.figure(figsize=(10,5))
# for i, key in enumerate(d2):
#     entry = d2[key]
#     Xs, Ys = entry.x, entry.y

#     C = location(Ys)
#     C = smooth.whittaker_smooth(C, tau2=1.0)
#     # As, Ls, Us = find_pc_all(Ys - C, 2)
#     As2, Ls2, Us2 = find_pc_smoothed_all(Ys - C, 10, tau=1.0)
#     # print(As2)
#     # plt.plot(Ls, marker='o', label=key)
#     plt.figure(figsize=(12,4))
#     # plt.plot(Ls_r, marker='o', label=key)
#     # plt.scatter(Us[0], Us[1], label=key)
#     plt.plot(As2[0], label=key+"-0")
#     plt.plot(As2[1], label=key+"-1")
#     plt.plot(As2[2], label=key+"-2")
#     plt.legend()
#     plt.show()

#     plt.scatter(Us2[0], Us2[1], label=key)
#     plt.tight_layout()
#     plt.legend()
#     plt.show()
#     plt.figure()
#     plt.scatter(Us2[1], Us2[2], label=key)
#     plt.tight_layout()
#     plt.legend()
#     plt.show()


In [ ]:
plt.plot(Ls1, marker='o', label='pca')
plt.plot(Ls2, marker='o', label='pca smoothed')
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
for i, A in enumerate(As1[:3]):
    plt.plot(A, label=str(i+1))
plt.legend()
plt.show()
plt.figure(figsize=(12,5))
for i, A in enumerate(As2[:3]):
    plt.plot(A, label=str(i+1))
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
for i, key in enumerate(d2):
    entry = d2[key]
    Xs, Ys = entry.x, entry.y

    # if "_R_" in key:
    #     if "95" in key:
    #         color='r'
    #     elif "96" in key:
    #         color='Brown'
    # elif "_S_" in key:
    #     if "92" in key:
    #         color='m'
    #     elif "93" in key:
    #         color='b'
    #     else:
    #         color='b'

    if "_R_" in key:
        color='r'
    elif "_S_" in key:
        color='b'
    else:
        color='m'
        
    # plt.plot(np.mean(Xs, axis=0), np.mean(Ys, axis=0), linewidth=1.0, color=color, label=key)
    plt.plot(np.mean(Ys, axis=0), linewidth=1.0, label=key)
    
# plt.hlines(0, 0, len(Ys[0]), colors='k')
# plt.xlim(-10,1000)
        
plt.minorticks_on()
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
import sympy
from sympy import MatrixSymbol, Identity, latex

In [ ]:
from IPython.display import display, Latex

In [ ]:
def latex_matrix(DD):
    n = DD.shape[0]
    s = r"$$\left(\begin{array}{" + n*"l" + "}"
    for i in range(10):
        for j in range(10):
            s += latex(DD[i,j])
            if j < n-1:
                s += r"&"
            else:
                s += r"\\"
    s += r"\end{array}\right)$$"
    # print(s)
    return s

In [ ]:
E = np.eye(10, dtype="i")
D2 = np.diff(E, 2, axis=0)
print(D2)
print(D2.T @ D2)

In [ ]:
w2 = sympy.symarray("w", (10,), )
W2 = np.diag(w2[0:-2])

In [ ]:
DD2 = np.dot(np.dot(D2.T, W2), D2)

In [ ]:
display(Latex(latex_matrix(DD2)))

In [ ]:
D1 = np.diff(np.eye(10, dtype="i"), 1, axis=0, )
print(D1)
print(D1.T @ D1)

In [ ]:
w = sympy.symarray("w", (10,))
W = np.diag(w[:-1])
DD1 = np.dot(np.dot(D1.T, W), D1)

In [ ]:
display(Latex(latex_matrix(DD1)))

In [ ]:
D3 = np.diff(np.eye(10, dtype="i"), 3, axis=0, )
print(D3)
print(D3.T @ D3)

In [ ]:
w3 = sympy.symarray("w", (10,), )
W3 = np.diag(w3[0:-3])

In [ ]:
DD3 = np.dot(np.dot(D3.T, W3), D3)

In [ ]:
display(Latex(latex_matrix(DD3)))

In [ ]:
D4 = np.diff(np.eye(10, dtype="i"), 4, axis=0, )
print(D4)
print(D4.T @ D4)

In [ ]:
w4 = sympy.symarray("w", (10,), )
W4 = np.diag(w4[0:-4])

In [ ]:
DD4 = np.dot(np.dot(D4.T, W4), D4)

In [ ]:
display(Latex(latex_matrix(DD4)))

In [ ]:
import ipywidgets as widgets
import os, time

In [ ]:
buttons = []
for i in range(10):
    buttons.append(widgets.Button(description=f"  The button {i}  "))
box = ipywidgets.Box(buttons)
box.layout = widgets.Layout(flex_flow="row wrap")
box

In [ ]:
fp = widgets.FloatProgress(min=0, max=10, value=0, bar_style="success", description="abcdefghijklmnopqrstuv")
fp.style.width=32
display(fp)
for i in range(11):
    time.sleep(1)
    fp.value=i